In [ ]:
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
db_file_path = 'ecommerce_data.db'
print(f"Database will be stored temporarily at: {db_file_path}")

Database will be stored temporarily at: ecommerce_data.db


In [ ]:
conn = sqlite3.connect(db_file_path)
cursor = conn.cursor()

In [ ]:
create_eligibility_sql = """
CREATE TABLE IF NOT EXISTS product_level_eligibility_table (
    item_id INTEGER NOT NULL,
    eligibility_datetime_utc DATETIME NOT NULL,
    eligibility BOOLEAN,
    message TEXT,
    PRIMARY KEY (item_id, eligibility_datetime_utc)
);
"""
cursor.execute(create_eligibility_sql)
print("Table 'product_level_eligibility_table' created or already exists.")

Table 'product_level_eligibility_table' created or already exists.


In [ ]:
create_ad_sales_sql = """
CREATE TABLE IF NOT EXISTS product_level_ad_sales_metrics (
    date DATE NOT NULL,
    item_id INTEGER NOT NULL,
    ad_sales REAL,
    impressions INTEGER,
    ad_spend REAL,
    clicks INTEGER,
    units_sold INTEGER,
    PRIMARY KEY (date, item_id)
);
"""
cursor.execute(create_ad_sales_sql)
print("Table 'product_level_ad_sales_metrics' created or already exists.")

Table 'product_level_ad_sales_metrics' created or already exists.


In [ ]:
create_total_sales_sql = """
CREATE TABLE IF NOT EXISTS product_level_total_sales_metrics (
    date DATE NOT NULL,
    item_id INTEGER NOT NULL,
    total_sales REAL,
    total_units_ordered INTEGER,
    PRIMARY KEY (date, item_id)
);
"""
cursor.execute(create_total_sales_sql)
print("Table 'product_level_total_sales_metrics' created or already exists.")

Table 'product_level_total_sales_metrics' created or already exists.


In [ ]:
conn.commit()
print("All table schemas committed to the database.")

All table schemas committed to the database.


In [ ]:
# --- Define your GitHub Raw URLs ---
eligibility_url = 'https://raw.githubusercontent.com/AyoAdi/Sales_chatbot/refs/heads/main/Datasets/Improved%20Datasets/product_level_eligibility_table_improved.csv'
ad_sales_url = 'https://raw.githubusercontent.com/AyoAdi/Sales_chatbot/refs/heads/main/Datasets/Improved%20Datasets/product_level_sales_and_metrics_improved.csv'
total_sales_url = 'https://raw.githubusercontent.com/AyoAdi/Sales_chatbot/refs/heads/main/Datasets/Improved%20Datasets/product_level_total_sales_and_metrics_improved.csv'

print("Loading cleaned DataFrames from GitHub URLs...")

# 1. Load and prepare Product-Level Eligibility Table
df_eligibility = pd.read_csv(eligibility_url)
df_eligibility['eligibility_datetime_utc'] = pd.to_datetime(df_eligibility['eligibility_datetime_utc'])
df_eligibility['eligibility'] = df_eligibility['eligibility'].astype(int)

# 2. Load and prepare Product-Level Ad Sales and Metrics
df_ad_sales = pd.read_csv(ad_sales_url)
df_ad_sales['date'] = pd.to_datetime(df_ad_sales['date'])

# 3. Load and prepare Product-Level Total Sales and Metrics
df_total_sales = pd.read_csv(total_sales_url)
df_total_sales['date'] = pd.to_datetime(df_total_sales['date'])

print("DataFrames loaded and prepped.")
print("Loading data into SQL tables...")

# --- FIX: Change 'if_exists' to 'replace' for initial load/testing ---
# This will drop the table if it exists and recreate it before inserting.
df_eligibility.to_sql(
    'product_level_eligibility_table',
    conn,
    if_exists='replace', # Changed from 'append' to 'replace'
    index=False
)
print("Data loaded into 'product_level_eligibility_table'.")

df_ad_sales.to_sql(
    'product_level_ad_sales_metrics',
    conn,
    if_exists='replace', # Changed from 'append' to 'replace'
    index=False
)
print("Data loaded into 'product_level_ad_sales_metrics'.")

df_total_sales.to_sql(
    'product_level_total_sales_metrics',
    conn,
    if_exists='replace', # Changed from 'append' to 'replace'
    index=False
)
print("Data loaded into 'product_level_total_sales_metrics'.")

conn.commit()
print("All data committed to the database.")

Loading cleaned DataFrames from GitHub URLs...
DataFrames loaded and prepped.
Loading data into SQL tables...
Data loaded into 'product_level_eligibility_table'.
Data loaded into 'product_level_ad_sales_metrics'.
Data loaded into 'product_level_total_sales_metrics'.
All data committed to the database.


## The tables are now created!! So the next step is to pick an LLM or an api key to handle the entire process

In [ ]:
!pip install -U google-generativeai

In [ ]:
import os
import pandas as pd
import sqlite3
import google.generativeai as genai
print("Libraries installed and imported.")

Libraries installed and imported.


In [ ]:
db_file_path = 'ecommerce_data.db'
print(f"Database will be stored temporarily at: {db_file_path}")

Database will be stored temporarily at: ecommerce_data.db


In [ ]:
conn = sqlite3.connect(db_file_path)
cursor = conn.cursor()
print("Connected to SQLite database.")

Connected to SQLite database.


In [ ]:
print("\nConfiguring Gemini Pro LLM...")
# Get API key from Colab Secrets
try:
    from google.colab import userdata
    # This correctly references the secret named 'GOOGLE_API_KEY'
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Gemini API key loaded from Colab Secrets and configured.")
except Exception as e:
    print(f"ERROR: Could not load Gemini API key from Colab Secrets. Please check secret name and access: {e}")
    print("Ensure your Gemini API key is stored as a Colab Secret named 'GOOGLE_API_KEY' and notebook access is enabled.")


Configuring Gemini Pro LLM...
Gemini API key loaded from Colab Secrets and configured.


In [ ]:
gemini_model = genai.GenerativeModel('models/gemini-2.0-flash')
print("Gemini Flash model initialized.")

Gemini Flash model initialized.


In [ ]:
def get_table_schema(cursor):
    """Fetches the schema of all tables in the SQLite database."""
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    schema_info = {}
    for table_name_tuple in tables:
        table_name = table_name_tuple[0]
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        column_details = []
        for col in columns:
            cid, name, ctype, notnull, dflt_value, pk = col
            column_details.append(f"{name} {ctype} {'NOT NULL' if notnull else ''} {'PRIMARY KEY' if pk else ''}")
        schema_info[table_name] = ", ".join(column_details)
    return schema_info

In [ ]:
def ask_llm_for_sql(question: str, schema: dict, gemini_model) -> str:
    """
    Asks the Gemini Pro LLM to convert a natural language question into an SQL query.
    """
    schema_str = "\n".join([f"Table: {table}\nColumns: {cols}" for table, cols in schema.items()])

    prompt_parts = [
        "You are an expert SQL query generator. Your sole purpose is to convert user questions into precise and accurate SQL queries.",
        "You must use the provided database schema to identify the correct tables and columns.",
        "Strictly adhere to the schema provided below.",
        "",
        "Here is the database schema:",
        schema_str,
        "",
        "**Important Rules for SQL Generation:**",
        "-   **Always select the correct table(s) based on the question's context.**",
        "    -   'total sales', 'units ordered' -> use `product_level_total_sales_metrics`",
        "    -   'ad sales', 'impressions', 'ad spend', 'clicks', 'units sold from ads', 'CPC', 'RoAS' -> use `product_level_ad_sales_metrics`",
        "    -   'eligible products', 'eligibility status' -> use `product_level_eligibility_table`",
        "-   Use appropriate SQL aggregate functions (SUM, AVG, MAX, MIN, COUNT) for questions involving totals, averages, highest/lowest, or counts.",
        "-   Use JOINs only when necessary to combine data from different tables based on common columns (like item_id or date).",
        "-   Filter data using WHERE clauses (e.g., for specific dates, item_ids, or eligibility status).",
        "-   When asked for 'Which product...' or 'Top X products...', use `GROUP BY item_id` with `ORDER BY` and `LIMIT`.",
        "-   Ensure column names and table names exactly match the schema.",
        "-   For RoAS, use `SUM(ad_sales) * 1.0 / SUM(ad_spend)` to ensure float division.",
        "-   For CPC, use `SUM(ad_spend) * 1.0 / SUM(clicks)` to ensure float division.",
        "-   **Output Format: STRICTLY ONLY THE SQL QUERY. DO NOT include any markdown code blocks (e.g., ```sql ```), introductory text, explanations, or concluding remarks.**",
        "",
        "Examples:",
        "User Question: What is my total sales?",
        "SQL Query: SELECT SUM(total_sales) FROM product_level_total_sales_metrics;",
        "",
        "User Question: Calculate the RoAS (Return on Ad Spend).",
        "SQL Query: SELECT SUM(ad_sales) * 1.0 / SUM(ad_spend) FROM product_level_ad_sales_metrics;",
        "",
        "User Question: Which product had the highest CPC (Cost Per Click)?",
        "SQL Query: SELECT item_id, SUM(ad_spend) * 1.0 / SUM(clicks) AS cpc FROM product_level_ad_sales_metrics GROUP BY item_id ORDER BY cpc DESC LIMIT 1;",
        "",
        "User Question: How many eligible products are there?",
        "SQL Query: SELECT COUNT(DISTINCT item_id) FROM product_level_eligibility_table WHERE eligibility = 1;",
        "",
        "User Question: What was the total units ordered on 2023-01-01?",
        "SQL Query: SELECT SUM(total_units_ordered) FROM product_level_total_sales_metrics WHERE date = '2023-01-01';",
        "",
        "User Question: Show me the ad sales for item 12345 on 2023-01-05.",
        "SQL Query: SELECT ad_sales FROM product_level_ad_sales_metrics WHERE item_id = 12345 AND date = '2023-01-05';",
        "",
        f"User Question: {question}",
        "SQL Query:"
    ]

    try:
        response = genai.GenerativeModel.generate_content(gemini_model, "\n".join(prompt_parts))
        generated_text = response.text.strip()

        # Refined parsing logic (same as before, it's good to keep this for robustness)
        if generated_text.lower().startswith("```sql"):
            generated_text = generated_text[len("```sql"):].strip()
        if generated_text.endswith("```"):
            generated_text = generated_text[:-len("```")].strip()
        if generated_text.startswith("```"):
            generated_text = generated_text[len("```"):].strip()

        sql_lines = generated_text.split('\n')
        final_sql = []
        in_sql_block = False
        for line in sql_lines:
            stripped_line_upper = line.strip().upper()
            if not in_sql_block and any(stripped_line_upper.startswith(kw) for kw in ["SELECT", "INSERT", "UPDATE", "DELETE", "CREATE", "DROP", "ALTER", "WITH", "PRAGMA"]):
                in_sql_block = True
                final_sql.append(line)
            elif in_sql_block:
                if stripped_line_upper == "" or (not any(char.isalpha() for char in stripped_line_upper) and ';' not in stripped_line_upper):
                    break
                final_sql.append(line)

        sql_query = "\n".join(final_sql).strip()

        if sql_query and not sql_query.endswith(';'):
            last_word = sql_query.split()[-1].upper() if sql_query.split() else ""
            if not any(last_word.endswith(kw) for kw in ["FROM", "WHERE", "BY", "LIMIT", "ON"]):
                 sql_query += ';'

        return sql_query
    except Exception as e:
        print(f"Error generating or parsing SQL from Gemini: {e}")
        return "ERROR: Failed to generate or parse SQL query from LLM."

In [ ]:
def execute_sql_query(cursor, sql_query):
    """Executes an SQL query and returns the results."""
    try:
        cursor.execute(sql_query)
        result = cursor.fetchall()
        column_names = [description[0] for description in cursor.description] if cursor.description else []
        return result, column_names
    except sqlite3.Error as e:
        print(f"SQL Error: {e}")
        return None, None


In [ ]:
def format_answer(question: str, result: list, column_names: list) -> str:
    """
    Formats the SQL query result into a human-readable string.
    """
    if result is None:
        return "I encountered an error while querying the database."
    if not result:
        return "I couldn't find any relevant data for that question."

    if len(result) == 1 and len(result[0]) == 1:
        value = result[0][0]
        if isinstance(value, (int, float)):
            return f"The {question.lower()} is: {value:,.2f}" if 'sales' in question.lower() or 'spend' in question.lower() else f"The {question.lower()} is: {value}"
        else:
            return f"The answer to '{question}' is: {value}"
    else:
        df_result = pd.DataFrame(result, columns=column_names)
        return f"Here is the data for '{question}':\n{df_result.to_string(index=False)}"

In [ ]:
# Main AI Agent Function
def ask_ai_agent(question: str, gemini_model, cursor) -> str:
    """
    Orchestrates the AI agent's process to answer a question.
    """
    print(f"\nUser: {question}")
    print("Fetching schema...")
    schema = get_table_schema(cursor)
    print("Schema fetched.")

    print("Asking LLM for SQL query...")
    sql_query = ask_llm_for_sql(question, schema, gemini_model)
    print(f"Generated SQL: {sql_query}")

    if sql_query.startswith("ERROR:"):
        return sql_query

    print("Executing SQL query...")
    query_result, column_names = execute_sql_query(cursor, sql_query)
    print("SQL query executed.")

    print("Formatting answer...")
    human_readable_answer = format_answer(question, query_result, column_names)
    print("Answer formatted.")

    return human_readable_answer

In [ ]:
%%writefile run_agent.py
# run_agent.py - This script runs your AI agent from the Colab Terminal

# --- Phase 0: Initial Setup and Imports ---
# These packages need to be installed in the terminal's environment.
# We'll use subprocess to ensure they are installed when the script runs.
import subprocess
import sys

def install_packages():
    print("Installing/Upgrading necessary Python packages...")
    try:
        # Use sys.executable to ensure pip is called from the correct Python environment
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "google-generativeai", "pandas"])
        print("Packages installed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error installing packages: {e}")
        sys.exit(1)

install_packages() # Run installation when the script starts

import pandas as pd
import sqlite3
import google.generativeai as genai
import os
import time

print("Libraries imported.")

# --- Phase 1: Database Setup and Connection ---

# Define Database File Path (Temporary Storage - will be deleted when session ends)
db_file_path = 'ecommerce_data.db'
print(f"Database will be stored temporarily at: {db_file_path}")

# Connect to the SQLite database file
conn = sqlite3.connect(db_file_path)
cursor = conn.cursor()
print("Connected to SQLite database.")


# --- Phase 2: Define and Create SQL Tables ---
create_eligibility_sql = """
CREATE TABLE IF NOT EXISTS product_level_eligibility_table (
    item_id INTEGER NOT NULL,
    eligibility_datetime_utc DATETIME NOT NULL,
    eligibility BOOLEAN,
    message TEXT,
    PRIMARY KEY (item_id, eligibility_datetime_utc)
);
"""
cursor.execute(create_eligibility_sql)
print("Table 'product_level_eligibility_table' created or already exists.")

create_ad_sales_sql = """
CREATE TABLE IF NOT EXISTS product_level_ad_sales_metrics (
    date DATE NOT NULL,
    item_id INTEGER NOT NULL,
    ad_sales REAL,
    impressions INTEGER,
    ad_spend REAL,
    clicks INTEGER,
    units_sold INTEGER,
    PRIMARY KEY (date, item_id)
);
"""
cursor.execute(create_ad_sales_sql)
print("Table 'product_level_ad_sales_metrics' created or already exists.")

create_total_sales_sql = """
CREATE TABLE IF NOT EXISTS product_level_total_sales_metrics (
    date DATE NOT NULL,
    item_id INTEGER NOT NULL,
    total_sales REAL,
    total_units_ordered INTEGER,
    PRIMARY KEY (date, item_id)
);
"""
cursor.execute(create_total_sales_sql)
print("Table 'product_level_total_sales_metrics' created or already exists.")

conn.commit()
print("All table schemas committed to the database.")


# --- Phase 3: Load Data from GitHub URLs into SQL Tables ---
eligibility_url = 'https://raw.githubusercontent.com/AyoAdi/Sales_chatbot/refs/heads/main/Datasets/Improved%20Datasets/product_level_eligibility_table_improved.csv'
ad_sales_url = 'https://raw.githubusercontent.com/AyoAdi/Sales_chatbot/refs/heads/main/Datasets/Improved%20Datasets/product_level_sales_and_metrics_improved.csv'
total_sales_url = 'https://raw.githubusercontent.com/AyoAdi/Sales_chatbot/refs/heads/main/Datasets/Improved%20Datasets/product_level_total_sales_and_metrics_improved.csv'

print("\nLoading cleaned DataFrames from GitHub URLs...")

df_eligibility = pd.read_csv(eligibility_url)
df_eligibility['eligibility_datetime_utc'] = pd.to_datetime(df_eligibility['eligibility_datetime_utc'])
df_eligibility['eligibility'] = df_eligibility['eligibility'].astype(int)

df_ad_sales = pd.read_csv(ad_sales_url)
df_ad_sales['date'] = pd.to_datetime(df_ad_sales['date'])

df_total_sales = pd.read_csv(total_sales_url)
df_total_sales['date'] = pd.to_datetime(df_total_sales['date'])

print("DataFrames loaded and prepped.")
print("Loading data into SQL tables...")

df_eligibility.to_sql( 'product_level_eligibility_table', conn, if_exists='replace', index=False)
print("Data loaded into 'product_level_eligibility_table'.")

df_ad_sales.to_sql( 'product_level_ad_sales_metrics', conn, if_exists='replace', index=False)
print("Data loaded into 'product_level_ad_sales_metrics'.")

df_total_sales.to_sql( 'product_level_total_sales_metrics', conn, if_exists='replace', index=False)
print("Data loaded into 'product_level_total_sales_metrics'.")

conn.commit()
print("All data committed to the database.")


# --- Phase 4: Configure and Use Gemini Flash LLM ---

print("\nConfiguring Gemini Flash LLM...")
# CORRECTED: Use os.getenv() to read API key from environment variable
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Gemini API key loaded from environment variable and configured.")
else:
    print("ERROR: GOOGLE_API_KEY environment variable not found.")
    print("Please set the GOOGLE_API_KEY environment variable in the terminal before running this script.")
    sys.exit(1) # Exit if API key is not found

# Initialize the Gemini Flash model
gemini_model = genai.GenerativeModel('models/gemini-2.5-flash')
print("Gemini Flash model initialized.")


# --- Phase 5: AI Agent Core Logic Functions ---

def get_table_schema(cursor):
    """Fetches the schema of all tables in the SQLite database."""
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    schema_info = {}
    for table_name_tuple in tables:
        table_name = table_name_tuple[0]
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        column_details = []
        for col in columns:
            cid, name, ctype, notnull, dflt_value, pk = col
            column_details.append(f"{name} {ctype} {'NOT NULL' if notnull else ''} {'PRIMARY KEY' if pk else ''}")
        schema_info[table_name] = ", ".join(column_details)
    return schema_info


def ask_llm_for_sql(question: str, schema: dict, gemini_model) -> str:
    """
    Asks the Gemini LLM to convert a natural language question into an SQL query.
    Returns the full generated SQL query string.
    """
    schema_str = "\n".join([f"Table: {table}\nColumns: {cols}" for table, cols in schema.items()])

    prompt_parts = [
        "You are an expert SQL query generator. Your sole purpose is to convert user questions into precise and accurate SQL queries.",
        "You must use the provided database schema to identify the correct tables and columns.",
        "Strictly adhere to the schema provided below.",
        "",
        "Here is the database schema:",
        schema_str,
        "",
        "**Important Rules for SQL Generation:**",
        "-   **Always select the correct table(s) based on the question's context.**",
        "    -   'total sales', 'units ordered' -> use `product_level_total_sales_metrics`",
        "    -   'ad sales', 'impressions', 'ad spend', 'clicks', 'units sold from ads', 'CPC', 'RoAS' -> use `product_level_ad_sales_metrics`",
        "    -   'eligible products', 'eligibility status' -> use `product_level_eligibility_table`",
        "-   Use appropriate SQL aggregate functions (SUM, AVG, MAX, MIN, COUNT) for questions involving totals, averages, highest/lowest, or counts.",
        "-   Use JOINs only when necessary to combine data from different tables based on common columns (like item_id or date).",
        "-   Filter data using WHERE clauses (e.g., for specific dates, item_ids, or eligibility status).",
        "-   When asked for 'Which product...' or 'Top X products...', use `GROUP BY item_id` with `ORDER BY` and `LIMIT`.",
        "-   Ensure column names and table names exactly match the schema.",
        "-   For RoAS, use `SUM(ad_sales) * 1.0 / SUM(ad_spend)` to ensure float division.",
        "-   For CPC, use `SUM(ad_spend) * 1.0 / SUM(clicks)` to ensure float division.",
        "-   **Output Format: STRICTLY ONLY THE SQL QUERY. DO NOT include any markdown code blocks (e.g., ```sql ```), introductory text, explanations, or concluding remarks.**",
        "",
        "Examples:",
        "User Question: What is my total sales?",
        "SQL Query: SELECT SUM(total_sales) FROM product_level_total_sales_metrics;",
        "",
        "User Question: Calculate the RoAS (Return on Ad Spend).",
        "SQL Query: SELECT SUM(ad_sales) * 1.0 / SUM(ad_spend) FROM product_level_ad_sales_metrics;",
        "",
        "User Question: Which product had the highest CPC (Cost Per Click)?",
        "SQL Query: SELECT item_id, SUM(ad_spend) * 1.0 / SUM(clicks) AS cpc FROM product_level_ad_sales_metrics GROUP BY item_id ORDER BY cpc DESC LIMIT 1;",
        "",
        f"User Question: {question}",
        "SQL Query:"
    ]

    full_generated_sql_text = ""
    try:
        response = gemini_model.generate_content("\n".join(prompt_parts))
        full_generated_sql_text = response.text.strip()

        generated_text = full_generated_sql_text.strip()

        if generated_text.lower().startswith("```sql"):
            generated_text = generated_text[len("```sql"):].strip()
        if generated_text.endswith("```"):
            generated_text = generated_text[:-len("```")].strip()
        if generated_text.startswith("```"):
            generated_text = generated_text[len("```"):].strip()

        sql_lines = generated_text.split('\n')
        final_sql = []
        in_sql_block = False
        for line in sql_lines:
            stripped_line_upper = line.strip().upper()
            if not in_sql_block and any(stripped_line_upper.startswith(kw) for kw in ["SELECT", "INSERT", "UPDATE", "DELETE", "CREATE", "DROP", "ALTER", "WITH", "PRAGMA"]):
                in_sql_block = True
                final_sql.append(line)
            elif in_sql_block:
                if stripped_line_upper == "" or (not any(char.isalpha() for char in stripped_line_upper) and ';' not in stripped_line_upper):
                    break
                final_sql.append(line)

        sql_query = "\n".join(final_sql).strip()

        if sql_query and not sql_query.endswith(';'):
            last_word = sql_query.split()[-1].upper() if sql_query.split() else ""
            if not any(last_word.endswith(kw) for kw in ["FROM", "WHERE", "BY", "LIMIT", "ON"]):
                 sql_query += ';'

        return sql_query
    except Exception as e:
        print(f"Error generating or parsing SQL from Gemini: {e}")
        return "ERROR: Failed to generate or parse SQL query from LLM."


def execute_sql_query(cursor, sql_query):
    """Executes an SQL query and returns the results."""
    try:
        cursor.execute(sql_query)
        result = cursor.fetchall()
        column_names = [description[0] for description in cursor.description] if cursor.description else []
        return result, column_names
    except sqlite3.Error as e:
        print(f"SQL Error: {e}")
        return None, None


def format_answer(question: str, result: list, column_names: list) -> str:
    """
    Formats the SQL query result into a human-readable string.
    """
    if result is None:
        return "I encountered an error while querying the database."
    if not result:
        return "I couldn't find any relevant data for that question."

    if len(result) == 1 and len(result[0]) == 1:
        value = result[0][0]
        if isinstance(value, (int, float)):
            return f"The {question.lower()} is: {value:,.2f}" if 'sales' in question.lower() or 'spend' in question.lower() else f"The {question.lower()} is: {value}"
        else:
            return f"The answer to '{question}' is: {value}"
    else:
        df_result = pd.DataFrame(result, columns=column_names)
        return f"Here is the data for '{question}':\n{df_result.to_string(index=False)}"


# --- Main AI Agent Function (for command-line interaction) ---
def ask_ai_agent_cli(question: str, gemini_model, cursor) -> str:
    """
    Orchestrates the AI agent's process to answer a question for CLI.
    """
    print(f"\nUser: {question}")
    print("Fetching schema...")
    schema = get_table_schema(cursor)
    print("Schema fetched.")

    print("Asking LLM for SQL query...")
    sql_query = ask_llm_for_sql(question, schema, gemini_model)
    print(f"Generated SQL: {sql_query}")

    if sql_query.startswith("ERROR:"):
        return sql_query

    print("Executing SQL query...")
    query_result, column_names = execute_sql_query(cursor, sql_query)
    print("SQL query executed.")

    print("Formatting answer...")
    human_readable_answer = format_answer(question, query_result, column_names)
    print("Answer formatted.")

    return human_readable_answer


# --- Interactive Command-Line Interface Loop (Entry Point) ---
if __name__ == "__main__": # This ensures the code below only runs when the script is executed directly
    # Get the schema once
    db_schema = get_table_schema(cursor)
    print("\n--- Database Schema ---")
    for table, cols in db_schema.items():
        print(f"{table}: {cols}")
    print("---------------------\n")

    print("--- AI Agent Command-Line Interface ---")
    print("Type your questions and press Enter. Type 'exit' to quit.")
    print("Examples: 'What is my total sales?', 'Calculate the RoAS (Return on Ad Spend).'")
    print("---------------------------------------\n")

    while True:
        user_question = input("You: ")
        if user_question.lower().strip() == 'exit':
            print("AI: Goodbye!")
            break

        answer = ask_ai_agent_cli(user_question, gemini_model, cursor)
        print(f"AI: {answer}\n")

    # Close the database connection when the loop exits
    conn.close()
    print("Database connection closed.")

Writing run_agent.py
